In [1]:
from sqlalchemy import create_engine, Table, MetaData, Column, String
import psycopg2
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
import os

# .dotenv configuration
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

db_user = os.getenv('USER')
db_pass = os.getenv('PASS')

# Connect to DB
engine = psycopg2.connect(database="auction", user = db_user, password = db_pass, 
                        host = "dbinstance.cekgqzi9pswg.us-east-1.rds.amazonaws.com", port = "5432")

con = engine.cursor()

con.execute('DROP TABLE IF EXISTS user_bid; DROP TABLE IF EXISTS bid; DROP TABLE IF EXISTS item; DROP TABLE IF EXISTS users')

# Create table users(id, username, password)
con.execute("""CREATE TABLE IF NOT EXISTS users(ID SERIAL PRIMARY KEY, username TEXT NOT NULL, password TEXT NOT NULL)""")

# Create table item(id, name, description, start_time, user_id)
con.execute("""CREATE TABLE IF NOT EXISTS item(id SERIAL PRIMARY KEY, name TEXT NOT NULL, description TEXT NOT NULL, 
               start_time TIMESTAMP WITHOUT TIME ZONE, user_id INTEGER NOT NULL, FOREIGN KEY(user_id) REFERENCES users)""")

# Create table bid(id, price, item_id)
con.execute("""CREATE TABLE IF NOT EXISTS bid(id SERIAL PRIMARY KEY, price NUMERIC NOT NULL, item_id INT NOT NULL, FOREIGN KEY(item_id) REFERENCES item(id))""")

# Create table user_bid(user_id, bid_id)
con.execute("""CREATE TABLE IF NOT EXISTS user_bid (user_id INT, bid_id INT, PRIMARY KEY(user_id,bid_id), FOREIGN KEY(user_id) REFERENCES users(id), FOREIGN KEY(bid_id) REFERENCES bid(id))""")

# Insert 3 values to users
con.execute("""INSERT INTO users VALUES (1, 'quan', 'quan'), (2, 'hai', 'hai'), (3, 'son', 'son')""")

# Insert 1 value to item
con.execute("""INSERT INTO item VALUES (1, 'pen', 'good status', TIMESTAMP '2015-05-18 15:02:11', 1)""")

# Insert 2 values to bid
con.execute("""INSERT INTO bid VALUES (1, 2.36, 1)""")
con.execute("""INSERT INTO user_bid VALUES (2, 1)""")

con.execute("""INSERT INTO bid VALUES (2, 2.55, 1)""")
con.execute("""INSERT INTO user_bid VALUES (3, 2)""")

# Find the highest price and whose price
con.execute("""SELECT ub.user_id, b.price FROM bid b INNER JOIN user_bid ub ON b.id = ub.bid_id WHERE b.item_id=1 AND b.price = (SELECT MAX(bid.price) FROM bid)""")

df = pd.DataFrame(con.fetchall())

con.execute('SELECT username FROM users WHERE users.ID=' + str(df.iloc[0, 0]))

df = pd.DataFrame(con.fetchall())

# Print out the username who has highest price
print("User " + str(df.iloc[0, 0]) + " won")

con.close()

User son won


In [68]:
# Import modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import re
import numpy as np

# Figures inline and set visualization style
%matplotlib inline
sns.set()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_train = pd.read_csv('Interview.csv', na_values=['NA', 'NaN', 'Na'])
df_train.head()

,Date of Interview,Client name,Industry,Location,Position to be closed,Nature of Skillset,Interview Type,Name(Cand ID),Gender,Candidate Current Location,Candidate Job Location,Interview Venue,Candidate Native location,Have you obtained the necessary permission to start at the required time,Hope there will be no unscheduled meetings,Can I Call you three hours before the interview and follow up on your attendance for the interview,Can I have an alternative number/ desk number. I assure you that I will not trouble you too much,Have you taken a printout of your updated resume. Have you read the JD and understood the same,Are you clear with the venue details and the landmark.,Has the call letter been shared,Expected Attendance,Observed Attendance,Marital Status,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 1,Male,Chennai,Hosur,Hosur,Hosur,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Single,NaN,NaN,NaN,NaN,NaN
1,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 2,Male,Chennai,Bangalore,Hosur,Trichy,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Single,NaN,NaN,NaN,NaN,NaN
2,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 3,Male,Chennai,Chennai,Hosur,Chennai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain,No,Single,NaN,NaN,NaN,NaN,NaN
3,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 4,Male,Chennai,Chennai,Hosur,Chennai,Yes,Yes,No,Yes,No,Yes,Yes,Uncertain,No,Single,NaN,NaN,NaN,NaN,NaN
4,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 5,Male,Chennai,Bangalore,Hosur,Chennai,Yes,Yes,Yes,No,Yes,Yes,Yes,Uncertain,No,Married,NaN,NaN,NaN,NaN,NaN


In [3]:
print(df_train.shape)
print(df_train.columns)
print(df_train.info())

(1234, 28)
Index(['Date of Interview', 'Client name', 'Industry', 'Location',
       'Position to be closed', 'Nature of Skillset', 'Interview Type',
       'Name(Cand ID)', 'Gender', 'Candidate Current Location',
       'Candidate Job Location', 'Interview Venue',
       'Candidate Native location',
       'Have you obtained the necessary permission to start at the required time',
       'Hope there will be no unscheduled meetings',
       'Can I Call you three hours before the interview and follow up on your attendance for the interview',
       'Can I have an alternative number/ desk number. I assure you that I will not trouble you too much',
       'Have you taken a printout of your updated resume. Have you read the JD and understood the same',
       'Are you clear with the venue details and the landmark.',
       'Has the call letter been shared', 'Expected Attendance',
       'Observed Attendance', 'Marital Status', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 2

In [69]:
df_train.drop([df_train.columns[i] for i in range(23, 28)], axis=1, inplace=True)
df_train.columns = ['date', 'name', 'industry', 'job_location', 'position', 'skillset', 'type', 'candidate_id', 'gender',
                    'candidate_address', 'candidate_job_location', 'venue', 'nation', 'is_ready', 'in_scheduled_meeting', 'prev_call',
                   'additional_member', 'userstand_jd', 'know_location', 'letter_received', 'expected_attendance', 'observed_attendance', 'marital_status']

df_train['date'] = pd.to_datetime(df_train['date'], errors='coerce')

df_train.drop([1233], inplace=True)

df_train.head()

,date,name,industry,job_location,position,skillset,type,candidate_id,gender,candidate_address,candidate_job_location,venue,nation,is_ready,in_scheduled_meeting,prev_call,additional_member,userstand_jd,know_location,letter_received,expected_attendance,observed_attendance,marital_status
0,2015-02-13,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 1,Male,Chennai,Hosur,Hosur,Hosur,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Single
1,2015-02-13,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 2,Male,Chennai,Bangalore,Hosur,Trichy,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Single
2,2015-02-13,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 3,Male,Chennai,Chennai,Hosur,Chennai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain,No,Single
3,2015-02-13,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 4,Male,Chennai,Chennai,Hosur,Chennai,Yes,Yes,No,Yes,No,Yes,Yes,Uncertain,No,Single
4,2015-02-13,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 5,Male,Chennai,Bangalore,Hosur,Chennai,Yes,Yes,Yes,No,Yes,Yes,Yes,Uncertain,No,Married


In [5]:
df_train.shape
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1233 entries, 0 to 1232
Data columns (total 23 columns):
date                      1211 non-null datetime64[ns]
name                      1233 non-null object
industry                  1233 non-null object
job_location              1233 non-null object
position                  1233 non-null object
skillset                  1233 non-null object
type                      1233 non-null object
candidate_id              1233 non-null object
gender                    1233 non-null object
candidate_address         1233 non-null object
candidate_job_location    1233 non-null object
venue                     1233 non-null object
nation                    1233 non-null object
is_ready                  1024 non-null object
in_scheduled_meeting      966 non-null object
prev_call                 966 non-null object
additional_member         967 non-null object
userstand_jd              966 non-null object
know_location             966 non-null obje

,date,name,industry,job_location,position,skillset,type,candidate_id,gender,candidate_address,candidate_job_location,venue,nation,is_ready,in_scheduled_meeting,prev_call,additional_member,userstand_jd,know_location,letter_received,expected_attendance,observed_attendance,marital_status
0,2015-02-13,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 1,Male,Chennai,Hosur,Hosur,Hosur,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Single
1,2015-02-13,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 2,Male,Chennai,Bangalore,Hosur,Trichy,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Single
2,2015-02-13,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 3,Male,Chennai,Chennai,Hosur,Chennai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain,No,Single
3,2015-02-13,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 4,Male,Chennai,Chennai,Hosur,Chennai,Yes,Yes,No,Yes,No,Yes,Yes,Uncertain,No,Single
4,2015-02-13,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 5,Male,Chennai,Bangalore,Hosur,Chennai,Yes,Yes,Yes,No,Yes,Yes,Yes,Uncertain,No,Married


In [21]:
assert pd.notnull(df_train['name']).all()

In [6]:
df_train['name'].value_counts()

Standard Chartered Bank            904
Hospira                             75
Pfizer                              75
Aon Hewitt                          28
Flextronics                         23
ANZ                                 22
Hewitt                              20
UST                                 18
Standard Chartered Bank Chennai     17
Prodapt                             17
Astrazeneca                         15
Williams Lea                        11
Barclays                             5
Aon hewitt Gurgaon                   2
Woori Bank                           1
Name: name, dtype: int64

In [7]:
df_train['industry'].value_counts()

BFSI                        949
Pharmaceuticals             165
IT Products and Services     45
Electronics                  23
IT Services                  23
Telecom                      17
IT                           11
Name: industry, dtype: int64

In [70]:
df_train.loc[df_train['industry']=='IT Services', 'industry'] = 'IT Products and Services'
df_train.loc[df_train['industry']=='IT', 'industry'] = 'IT Products and Services'
df_train['industry'].value_counts()

BFSI                        949
Pharmaceuticals             165
IT Products and Services     79
Electronics                  23
Telecom                      17
Name: industry, dtype: int64

In [9]:
df_train['job_location'].value_counts()

Chennai       754
Bangalore     292
chennai        86
Hyderabad      38
Gurgaon        33
Noida          15
- Cochin-       9
chennai         3
Delhi           1
Gurgaonr        1
CHENNAI         1
Name: job_location, dtype: int64

In [71]:
df_train.loc[df_train['job_location']=='chennai ', 'job_location'] = 'Chennai'
df_train.loc[df_train['job_location']=='CHENNAI', 'job_location'] = 'Chennai'
df_train.loc[df_train['job_location']=='Gurgaonr', 'job_location'] = 'Gurgaon'
df_train.loc[df_train['job_location']=='- Cochin- ', 'job_location'] = 'Cochin'
df_train['job_location'].value_counts()

Chennai      758
Bangalore    292
chennai       86
Hyderabad     38
Gurgaon       34
Noida         15
Cochin         9
Delhi          1
Name: job_location, dtype: int64

In [13]:
df_train['position'].value_counts()

Routine                1023
Niche                   163
Dot Net                  18
Trade Finance            11
AML                       8
Selenium testing          5
Production- Sterile       5
Name: position, dtype: int64

In [85]:
df_train['skillset'].value_counts()

In [72]:
pattern = '^[Java]'
mask = df_train['skillset'].str.contains(pattern)
df_train.loc[mask, 'skillset'] = 'JAVA/J2EE/Struts/Hibernate'

pattern = 'Java'
mask = df_train['skillset'].str.contains(pattern)
df_train.loc[mask, 'skillset'] = 'JAVA/J2EE/Struts/Hibernate'

In [73]:
pattern = '^[\d+]'
mask = df_train['skillset'].str.contains(pattern)
df_train.loc[mask, 'skillset'] = 'JAVA/J2EE/Struts/Hibernate'

df_train.loc[df_train['skillset']=='Oracle Plsql', 'skillset'] = 'Oracle'

In [16]:
df_train['type'].value_counts()

Scheduled Walk In    456
Scheduled            371
Scheduled Walkin     189
Walkin               189
Walkin                27
Sceduled walkin        1
Name: type, dtype: int64

In [74]:
df_train.loc[df_train['type']=='Sceduled walkin', 'type'] = 'Scheduled Walkin'
df_train.loc[df_train['type']=='Walkin ', 'type'] = 'Walkin'
df_train['type'].value_counts()

Scheduled Walk In    456
Scheduled            371
Walkin               216
Scheduled Walkin     190
Name: type, dtype: int64

In [76]:
df_train['gender'] = df_train['gender'].astype('category')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1233 entries, 0 to 1232
Data columns (total 23 columns):
date                      1211 non-null datetime64[ns]
name                      1233 non-null object
industry                  1233 non-null object
job_location              1233 non-null object
position                  1233 non-null object
skillset                  1233 non-null object
type                      1233 non-null object
candidate_id              1233 non-null object
gender                    1233 non-null category
candidate_address         1233 non-null object
candidate_job_location    1233 non-null object
venue                     1233 non-null object
nation                    1233 non-null object
is_ready                  1024 non-null object
in_scheduled_meeting      966 non-null object
prev_call                 966 non-null object
additional_member         967 non-null object
userstand_jd              966 non-null object
know_location             966 non-null ob

In [21]:
df_train['candidate_address'].value_counts()

Chennai       754
Bangalore     292
chennai        86
Hyderabad      38
Gurgaon        34
Noida          15
- Cochin-       9
chennai         3
Delhi           1
CHENNAI         1
Name: candidate_address, dtype: int64

In [77]:
df_train.loc[df_train['candidate_address']=='chennai', 'candidate_address']='Chennai'
df_train.loc[df_train['candidate_address']=='chennai ', 'candidate_address']='Chennai'
df_train.loc[df_train['candidate_address']=='CHENNAI', 'candidate_address']='Chennai'
df_train.loc[df_train['candidate_address']=='- Cochin- ', 'candidate_address']='Cochin'
df_train['candidate_address'].value_counts()

Chennai      844
Bangalore    292
Hyderabad     38
Gurgaon       34
Noida         15
Cochin         9
Delhi          1
Name: candidate_address, dtype: int64

In [78]:
df_train.loc[df_train['candidate_job_location']=='- Cochin- ', 'candidate_job_location']='Cochin'
df_train['candidate_job_location'].value_counts()

Chennai          893
Bangalore        259
Gurgaon           35
Visakapatinam     21
Noida             15
Cochin             9
Hosur              1
Name: candidate_job_location, dtype: int64

In [79]:
df_train.loc[df_train['venue']=='- Cochin- ', 'venue']='Cochin'
df_train['venue'].value_counts()

Chennai      852
Bangalore    277
Hyderabad     40
Gurgaon       35
Noida         15
Cochin         9
Hosur          5
Name: venue, dtype: int64

In [80]:
df_train.loc[df_train['nation']=='- Cochin- ', 'nation']='Cochin'
df_train['nation'].value_counts()

Chennai          595
Hyderabad        172
Bangalore        151
Cochin            29
Gurgaon           26
Cuttack           25
Pune              22
Coimbatore        21
Allahabad         20
Noida             17
Visakapatinam     16
Nagercoil         16
Kolkata           14
Trivandrum        14
Trichy            13
Vellore           12
Mumbai             7
Chitoor            6
Pondicherry        5
Chandigarh         5
Ahmedabad          5
Delhi              4
Tirupati           4
Vijayawada         4
Hosur              3
Salem              3
Warangal           3
Ambur              2
Delhi /NCR         2
Patna              2
Tanjore            1
Baddi              1
Hissar             1
Tuticorin          1
Belgaum            1
Kanpur             1
Ghaziabad          1
Mysore             1
Faizabad           1
Panjim             1
Bhubaneshwar       1
Kurnool            1
Anantapur          1
Agra               1
Lucknow            1
Name: nation, dtype: int64

In [81]:
df_train['is_ready'].value_counts()

Yes               917
No                 79
Not yet            19
yes                 4
Yet to confirm      4
NO                  1
Name: is_ready, dtype: int64

In [85]:
df_train.loc[df_train['is_ready']=='Not yet', 'is_ready']='No'
df_train.loc[df_train['is_ready']=='Yet to confirm', 'is_ready']=np.nan
df_train.loc[df_train['is_ready']=='yes', 'is_ready']='Yes'
df_train.loc[df_train['is_ready']=='NO', 'is_ready']='No'
df_train['is_ready'].value_counts(dropna=False)

Yes    921
NaN    213
No      99
Name: is_ready, dtype: int64

In [86]:
df_train.loc[df_train['is_ready']=='yes', 'is_ready']='Yes'
df_train['in_scheduled_meeting'].value_counts(dropna=False)

Yes         949
NaN         267
No            6
yes           5
Not sure      4
cant Say      1
Not Sure      1
Name: in_scheduled_meeting, dtype: int64

In [65]:
#df_train[['is_ready', 'in_scheduled_meeting', 'prev_call', 'additional_member', 'userstand_jd', 'know_location', 'letter_received', 'expected_attendance', 'observed_attendance', 'marital_status']] = df_train[['is_ready', 'in_scheduled_meeting', 'prev_call', 'additional_member', 'userstand_jd', 'know_location', 'letter_received', 'expected_attendance', 'observed_attendance', 'marital_status']].astype('category')
for col in df_train.columns[13:]:
    df_train[col] = df_train[col].astype('category')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1233 entries, 0 to 1232
Data columns (total 23 columns):
date                      1211 non-null datetime64[ns]
name                      1233 non-null object
industry                  1233 non-null object
job_location              1233 non-null object
position                  1233 non-null object
skillset                  1233 non-null object
type                      1233 non-null object
candidate_id              1233 non-null object
gender                    1233 non-null category
candidate_address         1233 non-null object
candidate_job_location    1233 non-null object
venue                     1233 non-null object
nation                    1233 non-null object
is_ready                  1024 non-null category
in_scheduled_meeting      966 non-null category
prev_call                 966 non-null category
additional_member         967 non-null category
userstand_jd              966 non-null category
know_location             966 n

In [67]:
df_train['is_ready'].value_counts()

Yes               917
No                 79
Not yet            19
yes                 4
Yet to confirm      4
NO                  1
Name: is_ready, dtype: int64

In [23]:
df_train.describe(include=['O'])

,date,name,industry,job_location,position,skillset,type,candidate_id,gender,candidate_address,candidate_job_location,venue,nation,is_ready,in_scheduled_meeting,prev_call,additional_member,userstand_jd,know_location,letter_received,expected_attendance,observed_attendance,marital_status
count,1233,1234,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1029,986,986,986,985,985,988,1228,1233,1233
unique,96,16,7,11,7,92,6,1233,2,10,7,7,46,7,7,5,6,8,7,12,7,8,2
top,06.02.2016,Standard Chartered Bank,BFSI,Chennai,Routine,JAVA/J2EE/Struts/Hibernate,Scheduled Walk In,Candidate 665,Male,Chennai,Chennai,Chennai,Chennai,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Single
freq,220,904,949,754,1023,220,456,1,965,754,893,852,595,917,949,951,936,940,946,932,882,701,767


In [30]:
df_train['date'] = pd.to_datetime(df_train['date'], errors='coerce')
df_train['date'].value_counts()

2016-06-02    220
2016-04-12     86
2016-04-13     84
2016-05-25     65
2016-04-21     61
2016-02-09     43
2015-06-25     43
2016-01-30     39
2016-01-22     34
2016-07-05     34
2015-03-04     30
2016-01-14     28
2016-02-27     26
2016-02-26     26
2016-08-20     25
2016-05-11     25
2016-02-23     22
2016-09-01     21
2016-11-05     19
2015-02-12     19
2015-06-19     18
2016-03-19     17
2015-03-17     16
2016-06-13     15
2016-03-06     14
2016-03-22     13
2015-03-13     13
2016-07-06     12
2015-04-04     11
2015-06-29     11
             ... 
2015-04-17      9
2016-10-05      9
2016-05-30      8
2016-04-22      8
2015-06-23      8
2015-05-02      7
2016-04-15      6
2016-04-23      5
2015-09-04      5
2016-05-05      5
2015-04-16      4
2015-04-21      4
2014-03-18      4
2016-08-05      3
2016-04-25      3
2016-06-05      3
2016-05-26      3
2016-04-20      2
2016-04-19      2
2016-05-24      2
2016-04-18      2
2023-12-04      1
2021-12-04      1
2016-04-05      1
2016-12-04

In [39]:
df_train['name'].value_counts(dropna=True)

Standard Chartered Bank            904
Pfizer                              75
Hospira                             75
Aon Hewitt                          28
Flextronics                         23
ANZ                                 22
Hewitt                              20
UST                                 18
Prodapt                             17
Standard Chartered Bank Chennai     17
Astrazeneca                         15
Williams Lea                        11
Barclays                             5
Aon hewitt Gurgaon                   2
Woori Bank                           1
﻿﻿                                   1
Name: name, dtype: int64

In [51]:
prep_name = df_train['name'].value_counts(dropna=True).keys()[-1]
print(prep_name)
df_train[df_train['name']==prep_name]

﻿﻿


,date,name,industry,job_location,position,skillset,type,candidate_id,gender,candidate_address,candidate_job_location,venue,nation,is_ready,in_scheduled_meeting,prev_call,additional_member,userstand_jd,know_location,letter_received,expected_attendance,observed_attendance,marital_status
1233,NaT,﻿﻿,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
?pd.DataFrame.dropna

Signature: pd.DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)
Docstring:
Return object with labels on given axis omitted where alternately any
or all of the data are missing

Parameters
----------
axis : {0 or 'index', 1 or 'columns'}, or tuple/list thereof
    Pass tuple or list to drop on multiple axes
how : {'any', 'all'}
    * any : if any NA values are present, drop that label
    * all : if all values are NA, drop that label
thresh : int, default None
    int value : require that many non-NA values
subset : array-like
    Labels along other axis to consider, e.g. if you are dropping rows
    these would be a list of columns to include
inplace : boolean, default False
    If True, do operation inplace and return None.

Returns
-------
dropped : DataFrame

Examples
--------
>>> df = pd.DataFrame([[np.nan, 2, np.nan, 0], [3, 4, np.nan, 1],
...                    [np.nan, np.nan, np.nan, 5]],
...                   columns=list('ABCD'))
>>> df
     A